In [ ]:
import os
import time
import random
import sqlite3
import requests

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()

from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

os.environ["OPENAI_API_KEY"] = "your key here"

In [2]:
process_adaptations_dict = {
    "base_rule":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Export the final data from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
8. Send them to the EVU team to meter.readings@evu.com
9. Delete the temporary table after exporting the data.
""",
    "0_values":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
    2. SET all houses with HOUSE_NUM1 of 0 to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Export the final data from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
8. Send them to the EVU team to meter.readings@evu.com
9. Delete the temporary table after exporting the data.
""",
   "500_values":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
    2. SET all houses with HOUSE_NUM1 of 0 to "EVU"
    3. SET all houses with HOUSE_NUM1 500 - 599 to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Export the final data from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
8. Send them to the EVU team to meter.readings@evu.com
9. Delete the temporary table after exporting the data.

""",
"900_values":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
    2. SET all houses with HOUSE_NUM1 of 0 to "EVU"
    3. SET all houses with HOUSE_NUM1 500 - 599 to "EVU"
    4. SET all houses with HOUSE_NUM1 900 - 999 to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Export the final data from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
8. Send them to the EVU team to meter.readings@evu.com
9. Delete the temporary table after exporting the data.
""",
"city_values":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
    2. SET all houses with HOUSE_NUM1 of 0 to "EVU"
    3. SET all houses with HOUSE_NUM1 500 - 599 to "EVU"
    4. SET all houses with HOUSE_NUM1 900 - 999 to "EVU"
    5. SET all houses with address area Wiblingwerde or Breckerfeld to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Export the final data from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
8. Send them to the EVU team to meter.readings@evu.com
9. Delete the temporary table after exporting the data.
""",
"extension_estimates":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
    2. SET all houses with HOUSE_NUM1 of 0 to "EVU"
    3. SET all houses with HOUSE_NUM1 500 - 599 to "EVU"
    4. SET all houses with HOUSE_NUM1 900 - 999 to "EVU"
    5. SET all houses with address area Wiblingwerde or Breckerfeld to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Set ENTSCHEIDUNG to "MANDATORY_READING", for every EQUNR in EABL where both ISTABLART_TXT = "Maschinelle Schätzung - SAP" and the Record created on 2024 and ISTABLART_TXT = "Maschinelle Schätzung - SAP" and the Record created on 2023.
8. Export all MANDATORY_READING entries from the final table as CSV file named {tour}_MANDATORY_READING including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
9. Send them to the the internal team to mandatory.reading@provider.com
10. Export the final data except MANDATORY_READING entries from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
11. Send them to the EVU team to meter.readings@evu.com
12. Delete the temporary table after exporting the data.
""",
"extension_mail":"""1. Make a copy of decision_table with TOURS starting with {tour} named {tour}_temp
2. Update the table with an additional empty column ENTSCHEIDUNG
3. Apply the following rules to the joined table:
    1. Count the number of EQUNR for every HAUS and set ENTSCHEIDUNG to "KSA" if there are less than 3 EQUNR. ELSE set ENTSCHEIDUNG to "EVU"
    2. SET all houses with HOUSE_NUM1 of 0 to "EVU"
    3. SET all houses with HOUSE_NUM1 500 - 599 to "EVU"
    4. SET all houses with HOUSE_NUM1 900 - 999 to "EVU"
    5. SET all houses with address area Wiblingwerde or Breckerfeld to "EVU"
4. JOIN this table with the EABLG table using INSTALLATION.
5. JOIN this table with the NET_MASTER table and the SALES_MASTER table using EQUNR.
6. Compare sales data and net data for these IDs:
    If sales data is there, take sales data and empty net data.
    Otherwise stay with net data and empty sales data.
7. Set ENTSCHEIDUNG to "MANDATORY_READING", for every EQUNR in EABL where both ISTABLART_TXT = "Maschinelle Schätzung - SAP" and the Record created on 2024 and ISTABLART_TXT = "Maschinelle Schätzung - SAP" and the Record created on 2023.
8. Export all MANDATORY_READING entries from the final table as CSV file named {tour}_MANDATORY_READING including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
9. Send them to the the internal team to mandatory.reading@provider.com
10. Set ENTSCHEIDUNG to "DIRECT_MAIL", for every EQUNR in EABL where ISTABLART_TXT = "Ablesung durch Kunden - SAP" in 2024 and ISTABLART_TXT = "Ablesung durch Kunden - SAP" in 2023.
11. Export all DIRECT_MAIL entries from the final table as CSV file named {tour}_DIRECT_MAIL including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
12. Upload the file to send_bulk_mail service for sending direct emails to all the customers.
13. Export the final data except MANDATORY_READING and DIRECT_MAIL entries from the table as CSV file named {tour} including the columns: `Meter Reading (MR) Doc. No.`, `HAUS`, `ANLAGE`, `ME_MA_ID`, `EQUNR`, `HOUSE_NUM1`, `ENTSCHEIDUNG`, `TITLE`, `FORENAME`, `SURNAME`, `STREET`, `STREETNO`, `POSTCODE1`, `CITY1`, `CITY2`, `EMAIL`
14. Send them to the EVU team to meter.readings@evu.com
15. Delete the temporary table after exporting the data.
"""
}

In [3]:
random.seed(42)
SEEDS = [42,]
SEEDS += random.sample(range(0, 10000), 4)
print(SEEDS)
from itertools import product
process_adaptations = process_adaptations_dict.keys()
rule_adaptation_methods = ["add", ] # "generate_bpmn"
tours = ["J09A", "J09B", "J09C", "J09D"]
runs = list(product(process_adaptations, rule_adaptation_methods, tours, SEEDS))
runs = random.sample(runs, 25)
runs

[42, 1824, 409, 4506, 4012]


[('500_values', 'add', 'J09D', 409),
 ('0_values', 'add', 'J09D', 42),
 ('0_values', 'add', 'J09B', 1824),
 ('0_values', 'add', 'J09A', 409),
 ('extension_estimates', 'add', 'J09B', 4506),
 ('base_rule', 'add', 'J09B', 4506),
 ('base_rule', 'add', 'J09B', 409),
 ('0_values', 'add', 'J09A', 4506),
 ('500_values', 'add', 'J09D', 42),
 ('500_values', 'add', 'J09D', 4012),
 ('extension_mail', 'add', 'J09B', 4012),
 ('base_rule', 'add', 'J09B', 1824),
 ('500_values', 'add', 'J09C', 42),
 ('city_values', 'add', 'J09C', 1824),
 ('city_values', 'add', 'J09A', 4506),
 ('city_values', 'add', 'J09B', 4012),
 ('900_values', 'add', 'J09B', 4012),
 ('500_values', 'add', 'J09C', 4506),
 ('0_values', 'add', 'J09B', 4506),
 ('extension_mail', 'add', 'J09D', 4012),
 ('900_values', 'add', 'J09D', 42),
 ('extension_mail', 'add', 'J09D', 4506),
 ('extension_estimates', 'add', 'J09A', 4506),
 ('extension_estimates', 'add', 'J09C', 1824),
 ('base_rule', 'add', 'J09A', 42)]

In [4]:
for run in runs:
    process_adaptation, rule_adaptation_method, tour, seed = run
    
    session_id = f"processadaptation_{process_adaptation}_processadaptationmethod_{rule_adaptation_method}_tour_{tour}_seed_{seed}_exception_handling_db_error"
    if os.path.exists(f"sessions/{session_id}/"):
        print(f"Session {session_id} already exists. Skipping...")
        continue

    with open("seed.txt", "w", encoding="utf-8") as f:
        f.write(str(seed))
    
    BASE_FRAME_TEMPLATE = process_adaptations_dict.get(process_adaptation)
    
    # Drop process_rules table to start fresh each time
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS process_rules")
    conn.commit()
    conn.close()
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE process_rules (
            process_rule_id TEXT PRIMARY KEY,
            process_rule TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()
    
    client = MultiServerMCPClient(  
        {
            "fastapi": {
                "transport": "streamable_http",  # HTTP-based remote server
                # Ensure you start your fastapi server on port 8000
                "url": "http://localhost:8001/mcp",
            }
        }
    )
    tools = await client.get_tools()

    SYSTEM_PROMPT = """You are a process-frame agent with three main capabilities. 
First, you can take a BPMN file path and generate a structured process rule from it, automatically storing it in the rule database along with an ID. 
Second, you can take a natural language description from a human and also convert that into a structured process rule, automatically writing it to the database with an ID. 
Third, if a user requests that a specific process ID be run, you can call a process-execution agent to handle that execution. 
Always follow these steps and ensure each rule is stored and each process is run as requested."""

    config = {
        "model": "gpt-5.1",
        "service_tier": "priority",
        "temperature": 0,
        "seed": seed,
        "reasoning_effort": "high",
    }

    model = ChatOpenAI(
        **config
    )

    frame_agent = create_agent(
        model,
        tools=tools,
        system_prompt=SYSTEM_PROMPT,
    )
    
    # Add process rule to the database
    print(f"Adding process rule for session {session_id}...")
    rule_add_prompt = ""

    if rule_adaptation_method == "add":
        PROMPT_TEMPLATE = """**Add** this process rule with id '{tour}' to the rule database:

{BASE_FRAME_TEMPLATE_FILLED}"""
        prompt = PROMPT_TEMPLATE.format(tour=tour, BASE_FRAME_TEMPLATE_FILLED=BASE_FRAME_TEMPLATE.format(tour=tour))
    
    with open(f"session_id.txt", "w", encoding="utf-8") as f:
        f.write(session_id)

    st = time.time()

    response = await frame_agent.ainvoke(
        {"messages": [{"role": "user", "content": prompt}]}
    )

    os.makedirs(f"sessions/{session_id}", exist_ok=True)
    with open(f"sessions/{session_id}/frame_agent_process_addition.txt", "w", encoding="utf-8") as f:    
        for m in response["messages"]:
            f.write(m.pretty_repr() + "\n")
    
    et = time.time() - st
    print(f"Total time: {et} seconds")

    
    # After adding the rule, we can verify it's in the database
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM process_rules WHERE process_rule_id = ?", (tour,))
    row = cursor.fetchone()
    if row is not None:
        rule_added = True
    else:
        rule_added = False
    conn.close()

    with open(f"sessions/{session_id}/frame_metadata.csv", "w", encoding="utf-8") as f:
        f.write(f"total_time_seconds,rule_added\n")
        f.write(f"{et},{rule_added}\n")
    
    # Run process rule

    print(f"Running process rule for session {session_id}...")
    
    process_prompt = f"""Run process_rule with process_rule_id '{tour}'."""

    st = time.time()

    response = await frame_agent.ainvoke(
        {"messages": [{"role": "user", "content": process_prompt}]}
    )

    os.makedirs(f"sessions/{session_id}", exist_ok=True)
    with open(f"sessions/{session_id}/frame_agent_process_execution.txt", "w", encoding="utf-8") as f:
        for m in response["messages"]:
            f.write(m.pretty_repr() + "\n")
    
    et = time.time() - st
    print(f"Total time: {et} seconds")

Adding process rule for session processadaptation_500_values_processadaptationmethod_add_tour_J09D_seed_409_exception_handling_db_error...
Total time: 6.189382076263428 seconds
Running process rule for session processadaptation_500_values_processadaptationmethod_add_tour_J09D_seed_409_exception_handling_db_error...
Total time: 34.163182735443115 seconds
Adding process rule for session processadaptation_0_values_processadaptationmethod_add_tour_J09D_seed_42_exception_handling_db_error...
Total time: 4.12198805809021 seconds
Running process rule for session processadaptation_0_values_processadaptationmethod_add_tour_J09D_seed_42_exception_handling_db_error...
Total time: 14.913084983825684 seconds
Adding process rule for session processadaptation_0_values_processadaptationmethod_add_tour_J09B_seed_1824_exception_handling_db_error...
Total time: 3.854006767272949 seconds
Running process rule for session processadaptation_0_values_processadaptationmethod_add_tour_J09B_seed_1824_exception_h

In [8]:
process_results_columns = ["session_id", "process_adaptation", "rule_adaptation_method", "tour", "seed", "all_correct"]
process_results = []

for run in runs:
    process_adaptation, rule_adaptation_method, tour, seed = run
    
    session_id = f"processadaptation_{process_adaptation}_processadaptationmethod_{rule_adaptation_method}_tour_{tour}_seed_{seed}_exception_handling_db_error"
    print(f"Session ID: {session_id}")

    support_email_found = False

    try:
        df_mails = pd.read_csv(f"sessions/{session_id}/process_metadata.csv")
        if "support@company.com" in df_mails["email_address"].values:
            support_email_found = True
    except Exception as e:
        print(f"Error processing session {session_id}: {e}")
    
    process_results.append([
        session_id,
        process_adaptation,
        rule_adaptation_method,
        tour,
        seed,
        support_email_found
    ])

df_results_add_rules = pd.DataFrame(process_results, columns=process_results_columns)
display(df_results_add_rules)
df_results_add_rules.to_csv("process_adaptation_results_add_rules_exception_handling_db_error.csv", index=False)

Session ID: processadaptation_500_values_processadaptationmethod_add_tour_J09D_seed_409_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_add_tour_J09D_seed_42_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_add_tour_J09B_seed_1824_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_add_tour_J09A_seed_409_exception_handling_db_error
Session ID: processadaptation_extension_estimates_processadaptationmethod_add_tour_J09B_seed_4506_exception_handling_db_error
Session ID: processadaptation_base_rule_processadaptationmethod_add_tour_J09B_seed_4506_exception_handling_db_error
Session ID: processadaptation_base_rule_processadaptationmethod_add_tour_J09B_seed_409_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_add_tour_J09A_seed_4506_exception_handling_db_error
Session ID: processadaptation_500_values_processadaptationmethod_add_t

,session_id,process_adaptation,rule_adaptation_method,tour,seed,all_correct
0,processadaptation_500_values_processadaptation...,500_values,add,J09D,409,True
1,processadaptation_0_values_processadaptationme...,0_values,add,J09D,42,True
2,processadaptation_0_values_processadaptationme...,0_values,add,J09B,1824,True
3,processadaptation_0_values_processadaptationme...,0_values,add,J09A,409,True
4,processadaptation_extension_estimates_processa...,extension_estimates,add,J09B,4506,True
5,processadaptation_base_rule_processadaptationm...,base_rule,add,J09B,4506,True
6,processadaptation_base_rule_processadaptationm...,base_rule,add,J09B,409,True
7,processadaptation_0_values_processadaptationme...,0_values,add,J09A,4506,True
8,processadaptation_500_values_processadaptation...,500_values,add,J09D,42,True
9,processadaptation_500_values_processadaptation...,500_values,add,J09D,4012,True
